In [1]:

import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv
/kaggle/working/__notebook__.ipynb


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

spaceship_Titanic =  pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
Submission_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
spaceship_Target = spaceship_Titanic['Transported']*1
spaceship_Data = spaceship_Titanic.drop('Transported',axis=1)
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

In [3]:
encoded_col = ['HomePlanet','CryoSleep','Destination','VIP','Cabin_deck','Cabin_side']
noencoded_col = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Cabin_num']
temp = spaceship_Data['Cabin'].str.split('/',expand=True)
temp.columns = ['Cabin_deck','Cabin_num','Cabin_side']
spaceship_data = pd.concat([spaceship_Data,temp],axis=1)
ordinal_encoder = OrdinalEncoder()
spaceship_cat_encoded = pd.DataFrame(ordinal_encoder.fit_transform(spaceship_data[encoded_col]))
spaceship_cat_encoded.columns = encoded_col
spaceship_prepro = pd.concat([spaceship_data[noencoded_col],spaceship_cat_encoded],axis=1)
X_train,X_test,y_train,y_test = train_test_split(spaceship_prepro,spaceship_Target,test_size=0.2)
X_train = num_pipeline.fit_transform(X_train)
X_test = num_pipeline.fit_transform(X_test)

In [4]:
param_grid =[
    {'kernel':['linear'], 'C': [10.,30.,100.,300.,1000.]},
    {'kernel':['rbf'], 'C': [1.0,3.0,10.],
     'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]}
]

svm_cls = SVC(kernel='rbf',C=30,gamma=0.03)
svm_cls.fit(X_train,y_train)
# grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error',verbose=2)
# grid_search.fit(X_train, y_train)

SVC(C=30, gamma=0.03)

In [5]:
Submission_feature = Submission_data.drop('PassengerId',axis=1)
encoded_col = ['HomePlanet','CryoSleep','Destination','VIP','Cabin_deck','Cabin_side']
noencoded_col = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Cabin_num']
temp = Submission_feature['Cabin'].str.split('/',expand=True)
temp.columns = ['Cabin_deck','Cabin_num','Cabin_side']
Submission_feature = pd.concat([Submission_feature,temp],axis=1)
ordinal_encoder = OrdinalEncoder()
submission_cat_encoded = pd.DataFrame(ordinal_encoder.fit_transform(Submission_feature[encoded_col]))
submission_cat_encoded.columns = encoded_col
submission_prepro = pd.concat([Submission_feature[noencoded_col],submission_cat_encoded],axis=1)
submission_prepared = num_pipeline.fit_transform(submission_prepro)

In [6]:
prediction = pd.concat([Submission_data['PassengerId'],pd.DataFrame(svm_cls.predict(submission_prepared))],axis=1)
prediction

,PassengerId,0
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,0
...,...,...
4272,9266_02,1
4273,9269_01,1
4274,9271_01,1
4275,9273_01,1


In [7]:
prediction.columns = ['PassengerId','Transported']
prediction['Transported'] = prediction['Transported'].astype('bool')
prediction

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [8]:
prediction.to_csv('My_output.csv',index=False)